# gevent

Source: http://www.gevent.org/contents.html

## Examples

In [1]:
import gevent
from gevent import socket
urls = ['www.google.com', 'www.example.com', 'www.python.org']
# 使用 gevent.socket 获取域名解析信息
jobs = [gevent.spawn(socket.gethostbyname, url) for url in urls]
gevent.joinall(jobs, timeout=2)
[job.value for job in jobs]

['93.46.8.89', '93.184.216.34', '151.101.88.223']

## Monkey patching

加速原有的 urllib2 模块

In [2]:
from gevent import monkey; monkey.patch_socket()
import urllib2 # it's usable from multiple greenlets now

## Beyond sockets

此功能未执行成功

In [ ]:
# from gevent import monkey; monkey.patch_all()
# import subprocess # it's usable from multiple greenlets now

## Timeout && Pool

解析多站点的例子; 组合 TimeOut(2秒钟) 和 Pool (10) 的例子. 

(see example: dns_mass_resolve.py).

In [ ]:
from __future__ import print_function
import gevent
from gevent import socket
from gevent.pool import Pool

# 解析 N 个站点
N = 1000

# 限制请求数 limit ourselves to max 10 simultaneous outstanding requests
pool = Pool(10)
finished = 0


def job(url):
    global finished
    try:
        try:
            ip = socket.gethostbyname(url)
            print('%s = %s' % (url, ip))
        except socket.gaierror as ex:
            print('%s failed with %s' % (url, ex))
    finally:
        finished += 1

# 看2秒钟内可以完成多少任务
with gevent.Timeout(2, False):
    for x in range(10, 10 + N):
        pool.spawn(job, '%s.com' % x)
    pool.join()
    
print('finished within 2 seconds: %s/%s' % (finished, N))